In [ ]:
#mansur

import os
from google.colab import files
files.upload()

os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')
!pip install kagglehub tensorflow transformers


In [ ]:
import kagglehub


dataset_path = kagglehub.dataset_download("jenlooper/language-of-flowers")
print("Path to dataset files:", dataset_path)


# List all files in the dataset directory
files = os.listdir(dataset_path)
print("Files in the dataset:", files)

# Check if any JSON files exist in the dataset
json_files = [f for f in files if f.endswith(".json")]
if json_files:
    print("JSON files found in the dataset:", json_files)
else:
    print("No JSON files found in the dataset.")


100%|██████████| 2.16k/2.16k [00:00<00:00, 4.02MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jenlooper/language-of-flowers/versions/2
Files in the dataset: ['language-of-flowers.csv']
No JSON files found in the dataset.


In [ ]:
!pip install tensorflow
!pip install transformers
!pip install datasets
!pip install faiss-cpu  # for CPU-only version


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.5 MB/s eta 0:00:00


In [ ]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from transformers import RagTokenizer




In [ ]:

import re
import requests
import pandas as pd
from transformers import pipeline, RagTokenForGeneration, RagTokenizer, RagRetriever
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load dataset (CSV file)
dataset_path = "/root/.cache/kagglehub/datasets/jenlooper/language-of-flowers/versions/2/language-of-flowers.csv"
data = pd.read_csv(dataset_path)

# Create a dictionary to map flower names to meanings
flower_info_dict = dict(zip(data['Flower'], data['Meaning']))

# Initialize the RAG model components
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", index_name="exact", use_dummy_dataset=True)
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)
tokenizer.pad_token_id=0
# Define the RAG pipeline for text generation
rag_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load GPT-2 model and tokenizer for text generation
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define GPT-2 text generation pipeline
gpt2_pipeline = pipeline("text-generation", model=gpt2_model, tokenizer=gpt2_tokenizer)

# Function to get flower information based on the flower name
def generate_flower_info(flower_name):
    """
    Generate information about a flower based on the flower name.

    Parameters:
        flower_name (str): Name of the flower.

    Returns:
        tuple: Flower name, flower description, and generated information.
    """
    # Get flower description from the dataset
    flower_description = flower_info_dict.get(flower_name, "No description available.")

    # Use GPT-2 model if we want to generate additional details about the flower
    query = f"Why is the flower {flower_name} associated with the meaning '{flower_description}'? Explain the cultural or historical significance behind the {flower_name}."
    # Ensure truncation is activated to avoid warnings
    generated_info = gpt2_pipeline(query, max_length=200, truncation=True)[0]["generated_text"]
    # Split the generated text into sentences (by punctuation)
    sentences = re.split(r'(?<=\w[.!?])\s+', generated_info.strip())

    # Limit the output to the first 5 sentences
    limited_output = "\n".join(sentences[:5])

    return flower_name, flower_description, limited_output





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Use the flower name directly for prediction
flower_name = "Aloe"  # This is the name of the flower you want to generate information for

# Get the flower's name, description, and generated info
predicted_flower, flower_description, generated_info = generate_flower_info(flower_name)

# Print the results
print("Predicted Flower:", predicted_flower)

print("Flower Description:", flower_description)

print("Generated Information:", generated_info)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted Flower: Aloe
Flower Description: Affection, also grief
Generated Information: Why is the flower Aloe associated with the meaning 'Affection, also grief'? Explain the cultural or historical significance behind the Aloe.
The Aloe is sacred, and it is one of its oldest relics and its greatest ornament, as a symbol of the love of the Goddess.
The meaning of Aloe's colour varies.
A pink has always been associated with joy, but in the Middle Ages it was applied for protection.
A brown is known for mourning; it symbolizes the loss of innocence.


In [ ]:
data.head()

,Color,Flower,Meaning
0,NaN,Abatina,Fickleness
1,NaN,Acanthus,"The fine art, artifice"
2,NaN,Aloe,"Affection, also grief"
3,NaN,Amaryllis,Pride
4,NaN,Anemone,"Forsaken, sickness"
